In [1]:
from typing import Tuple

import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'italy'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

model = Prophet().fit(df_train)

03:00:32 - cmdstanpy - INFO - Chain [1] start processing
03:00:32 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,8.822497,8.333357,9.377204,8.822497,8.822497,0.022789,0.022789,0.022789,0.022789,0.022789,0.022789,0.0,0.0,0.0,8.845286
1,2022-04-01,8.781602,8.323629,9.317883,8.781602,8.781602,0.012083,0.012083,0.012083,0.012083,0.012083,0.012083,0.0,0.0,0.0,8.793685
2,2022-05-01,8.742026,8.249918,9.283412,8.742026,8.742026,0.048477,0.048477,0.048477,0.048477,0.048477,0.048477,0.0,0.0,0.0,8.790503
3,2022-06-01,8.701130,8.153465,9.239812,8.700116,8.701130,0.013395,0.013395,0.013395,0.013395,0.013395,0.013395,0.0,0.0,0.0,8.714526
4,2022-07-01,8.661554,8.145509,9.161636,8.654646,8.662672,-0.025548,-0.025548,-0.025548,-0.025548,-0.025548,-0.025548,0.0,0.0,0.0,8.636006
5,2022-08-01,8.620659,8.021116,9.081506,8.606896,8.630429,-0.052422,-0.052422,-0.052422,-0.052422,-0.052422,-0.052422,0.0,0.0,0.0,8.568237
6,2022-09-01,8.579764,8.056484,9.147270,8.555710,8.598726,0.042609,0.042609,0.042609,0.042609,0.042609,0.042609,0.0,0.0,0.0,8.622373
7,2022-10-01,8.540188,8.078766,9.153889,8.500165,8.566230,0.081079,0.081079,0.081079,0.081079,0.081079,0.081079,0.0,0.0,0.0,8.621267
8,2022-11-01,8.499293,8.068126,9.142836,8.440605,8.533091,0.082588,0.082588,0.082588,0.082588,0.082588,0.082588,0.0,0.0,0.0,8.581881
9,2022-12-01,8.459716,7.946910,9.008973,8.386151,8.500831,0.040828,0.040828,0.040828,0.040828,0.040828,0.040828,0.0,0.0,0.0,8.500544


In [11]:
rmse

0.6039928564466459

In [12]:
with open(f'{country}_prophet_base_model.json', 'w') as f:
    f.write(model_to_json(model))

In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [14]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,8.340988,7.893775,8.932646,8.340988,8.340988,0.063397,0.063397,0.063397,0.063397,0.063397,0.063397,0.0,0.0,0.0,8.404386
1,2023-04-01,8.300093,7.776296,8.834615,8.300093,8.300093,0.011367,0.011367,0.011367,0.011367,0.011367,0.011367,0.0,0.0,0.0,8.311460
2,2023-05-01,8.260517,7.800502,8.872938,8.260517,8.260517,0.067926,0.067926,0.067926,0.067926,0.067926,0.067926,0.0,0.0,0.0,8.328443
